In [57]:
import os
import pickle
import functools
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from multiprocessing import Pool
from scipy.spatial.distance import cosine
from ph_likes.util import queries
from ph_likes.util import settings

# Load Data

In [2]:
# tables
df_post = queries.get_post_table()

In [3]:
CF = np.load(os.path.join(settings.LOCAL_MODELS_PATH, 'CF.npy'))
# post_ids
post_ids2index = pickle.load(open(os.path.join(settings.LOCAL_MODELS_PATH, 'post_ids2index.pkl'), 'rb'))
index2post_ids = pickle.load(open(os.path.join(settings.LOCAL_MODELS_PATH, 'index2post_ids.pkl'), 'rb'))
#
user_ids2index = pickle.load(open(os.path.join(settings.LOCAL_MODELS_PATH, 'user_ids2index.pkl'), 'rb'))
index2user_ids = pickle.load(open(os.path.join(settings.LOCAL_MODELS_PATH, 'index2user_ids.pkl'), 'rb'))

# Get Recommendation

In [51]:
def get_item_similarity(cf, item_index_1, item_index_2):
    v1 = cf[:, item_index_1]
    v2 = cf[:, item_index_2]
#     return 1.0 - cosine(v1, v2)
    return np.bitwise_and(v1, v2).sum()

In [52]:
USER_ID = settings.EXAMPLE_USER_ID
USER_ID_INDEX = user_ids2index[USER_ID]
USER_ID_INDEX

890

In [53]:
liked_item_indices = np.where(CF[890, :])[0]
not_liked_item_indices = np.array(list(set(range(CF.shape[1])) - set(liked_item_indices)))

In [ ]:
ITEM_SIMILARITIES_FOR_USER = dict()
def get_avg_similarity(item_index, liked_item_indices, cf):
    avg_similarity = np.mean([get_item_similarity(cf, item_index, i) for i in liked_item_indices])
    return avg_similarity

p_list = not_liked_item_indices
p_func = functools.partial(get_avg_similarity, liked_item_indices=liked_item_indices, cf=CF)
with Pool() as p:
    for avg_similarity in tqdm_notebook(p.imap(p_func, p_list), total=len(not_liked_item_indices)):
        ITEM_SIMILARITIES_FOR_USER[item_index] = avg_similarity